# Keras Deep Neural Network - Classification


This data set is made available by Universita di Bologna. It contains data about smarthphone sensors as people are using different modes of transportation. Each row represents a specific mode of transportation and the corresponding readings in the smartphone sensors (for that mode of transportation). Multiclass classification task: predict whether the mode of transportation based on a smartphone's sensor readings. Transportation mode prediction can provide context information to enhance applications and provide a better user experience. It can be crucial for many different applications, such as device profiling, monitoring road and traffic conditions, healthcare, travel support etc.

# Read and Prepare the Data

In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

transportation = pd.read_csv("transportation.csv")
transportation.head()

,time,androidsensoraccelerometermean,androidsensoraccelerometermin,androidsensoraccelerometermax,androidsensoraccelerometerstd,androidsensorgyroscopemean,androidsensorgyroscopemin,androidsensorgyroscopemax,androidsensorgyroscopestd,soundmean,soundmin,soundmax,soundstd,target
0,78,9.811476,9.758895,9.849411,0.014626,0.001651,0.000000,0.003533,0.000737,0.000000,0.000000,0.000000,0.000000,Still
1,145,9.939207,7.707437,17.146631,1.775944,0.036326,0.011669,0.059388,0.020290,89.200210,89.065143,89.335277,0.191013,Car
2,150,9.827178,9.804817,9.849262,0.011199,0.001525,0.000000,0.002609,0.000713,0.000000,0.000000,0.000000,0.000000,Still
3,105,9.673039,7.659674,12.304298,0.862553,0.036440,0.020463,0.052512,0.010553,87.470377,87.470377,87.470377,2.284186,Car
4,77,9.993466,8.965621,10.891645,0.504117,0.183202,0.020667,0.380875,0.098819,89.770732,89.770732,89.770732,0.006389,Car


In [2]:
#Check for missing values
transportation.isna().sum()

time                              0
androidsensoraccelerometermean    0
androidsensoraccelerometermin     0
androidsensoraccelerometermax     0
androidsensoraccelerometerstd     0
androidsensorgyroscopemean        0
androidsensorgyroscopemin         0
androidsensorgyroscopemax         0
androidsensorgyroscopestd         0
soundmean                         0
soundmin                          0
soundmax                          0
soundstd                          0
target                            0
dtype: int64

In [3]:
#Split into train/test
from sklearn.model_selection import train_test_split

train, test = train_test_split(transportation, test_size=0.3)

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

#Separate Target Variable
train_target = train[['target']]
test_target = test[['target']]

train_inputs = train.drop(['target'], axis=1)
test_inputs = test.drop(['target'], axis=1)

### Numeric/Categorical Pipeline

In [5]:
train_inputs.dtypes

time                                int64
androidsensoraccelerometermean    float64
androidsensoraccelerometermin     float64
androidsensoraccelerometermax     float64
androidsensoraccelerometerstd     float64
androidsensorgyroscopemean        float64
androidsensorgyroscopemin         float64
androidsensorgyroscopemax         float64
androidsensorgyroscopestd         float64
soundmean                         float64
soundmin                          float64
soundmax                          float64
soundstd                          float64
dtype: object

In [6]:
# Identify the numerical columns -**all columns are numeric
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns

numeric_columns

Index(['time', 'androidsensoraccelerometermean',
       'androidsensoraccelerometermin', 'androidsensoraccelerometermax',
       'androidsensoraccelerometerstd', 'androidsensorgyroscopemean',
       'androidsensorgyroscopemin', 'androidsensorgyroscopemax',
       'androidsensorgyroscopestd', 'soundmean', 'soundmin', 'soundmax',
       'soundstd'],
      dtype='object')

In [7]:
# Pipeline Operations
numeric_transformer = Pipeline(steps=[
                #('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns)])

### Transform Train and Test

In [8]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x.shape

(4125, 13)

In [9]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x.shape

(1768, 13)

### Determine Baseline Accuracy

In [10]:
# Find the majority class:
train_target['target'].value_counts()

Bus        870
Train      838
Car        824
Still      799
Walking    794
Name: target, dtype: int64

In [11]:
#Find the percentage of the majority class:
train_target['target'].value_counts()/len(train_target)

Bus        0.210909
Train      0.203152
Car        0.199758
Still      0.193697
Walking    0.192485
Name: target, dtype: float64

Baseline accuracy predicting Bus is 21.1%

### Ordinal Encode Target

In [12]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()

train_y = ord_enc.fit_transform(train_target)
train_y

array([[3.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [2.]])

In [13]:
test_y = ord_enc.transform(test_target)
test_y

array([[0.],
       [0.],
       [0.],
       ...,
       [2.],
       [3.],
       [2.]])

# Keras DNN model 1

In [20]:
import tensorflow as tf
from tensorflow import keras

tf.random.set_seed(42)

#Define multi-class ordinal model
model = keras.models.Sequential()

#Define input layer - 13 vars
model.add(keras.layers.Input(shape=13))

#Define hidden layers
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))

#Define output layer: softmax - 5 nodes
model.add(keras.layers.Dense(5, activation='softmax'))

#Optimizer:
adam = keras.optimizers.Adam(lr=0.01)

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [21]:
# Fit the model
history = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=50, batch_size=1000)

Epoch 1/50
5/5 [==============================] - 0s 22ms/step - loss: 1.3823 - accuracy: 0.4337 - val_loss: 1.1499 - val_accuracy: 0.5549
Epoch 2/50
5/5 [==============================] - 0s 7ms/step - loss: 1.0985 - accuracy: 0.5661 - val_loss: 0.9984 - val_accuracy: 0.6018
Epoch 3/50
5/5 [==============================] - 0s 6ms/step - loss: 0.9775 - accuracy: 0.6000 - val_loss: 0.9383 - val_accuracy: 0.6312
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 0.9072 - accuracy: 0.6301 - val_loss: 0.8523 - val_accuracy: 0.6686
Epoch 5/50
5/5 [==============================] - 0s 6ms/step - loss: 0.8578 - accuracy: 0.6514 - val_loss: 0.8480 - val_accuracy: 0.6629
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - loss: 0.8319 - accuracy: 0.6693 - val_loss: 0.7991 - val_accuracy: 0.6816
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 0.7965 - accuracy: 0.6742 - val_loss: 0.7920 - val_accuracy: 0.6765
Epoch 8/50
5/5 [=================

In [22]:
# Evaluate the model
scores = model.evaluate(test_x, test_y, verbose=0)
scores

#first # is loss, second # is accuracy

[0.6357447504997253, 0.8059954643249512]

In [23]:
# extract the accuracy from model.evaluate
print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.64
accuracy: 80.60%


# Keras DNN model 2

In [117]:
#Set learning rate
lr=0.001

#Available optimizers
adagrad = keras.optimizers.Adagrad(lr=lr, epsilon=None, decay=0.0)
sgd = keras.optimizers.SGD(lr=lr, momentum=0.0, decay=0.0, nesterov=False)
rmsprop = keras.optimizers.RMSprop(lr=lr, rho=0.9, epsilon=None, decay=0.0)
adam = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
nesterov_adam = keras.optimizers.Nadam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

#Initializers
xavier = keras.initializers.glorot_normal(seed=None)
he = keras.initializers.he_normal(seed=None)

# Activation functions
activation = 'elu' 
#activation = 'relu'
#activation = 'tanh'
#activation = 'sigmoid'

#define input layer
input1 = keras.layers.Input(shape=13)

#define hidden and dropout layers
hidden1 = keras.layers.Dense(100, activation=activation, kernel_initializer=he)(input1)
drop1   = keras.layers.Dropout(0.2)(hidden1)
hidden2 = keras.layers.Dense(100, activation=activation, kernel_initializer=he)(drop1)
drop2   = keras.layers.Dropout(0.2)(hidden2)
hidden3 = keras.layers.Dense(100, activation=activation, kernel_initializer=he)(drop2)
hidden4 = keras.layers.Dense(100, activation=activation, kernel_initializer=he)(hidden3)
hidden5 = keras.layers.Dense(100, activation=activation, kernel_initializer=he)(hidden4)

#define output layer - softmax 5 nodes
output = keras.layers.Dense(5, activation='softmax')(hidden5)

#compile model
model = keras.Model(inputs = input1, outputs = output)

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=nesterov_adam, metrics=['accuracy'])

In [118]:
# Fit the model
history = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=50, batch_size=100)

Epoch 1/50
42/42 [==============================] - 0s 5ms/step - loss: 1.4205 - accuracy: 0.4795 - val_loss: 0.9712 - val_accuracy: 0.6318
Epoch 2/50
42/42 [==============================] - 0s 3ms/step - loss: 1.0515 - accuracy: 0.5748 - val_loss: 0.9233 - val_accuracy: 0.6239
Epoch 3/50
42/42 [==============================] - 0s 3ms/step - loss: 0.9901 - accuracy: 0.5944 - val_loss: 0.8733 - val_accuracy: 0.6669
Epoch 4/50
42/42 [==============================] - 0s 3ms/step - loss: 0.9580 - accuracy: 0.6097 - val_loss: 0.8427 - val_accuracy: 0.6606
Epoch 5/50
42/42 [==============================] - 0s 3ms/step - loss: 0.9273 - accuracy: 0.6213 - val_loss: 0.7944 - val_accuracy: 0.6821
Epoch 6/50
42/42 [==============================] - 0s 3ms/step - loss: 0.9035 - accuracy: 0.6320 - val_loss: 0.7905 - val_accuracy: 0.6793
Epoch 7/50
42/42 [==============================] - 0s 3ms/step - loss: 0.8598 - accuracy: 0.6521 - val_loss: 0.7673 - val_accuracy: 0.6776
Epoch 8/50
42/42 [==

In [119]:
# Evaluate the model
scores = model.evaluate(test_x, test_y, verbose=0)
scores

[0.5107569098472595, 0.8065611124038696]

In [120]:
# extract the accuracy from model.evaluate
print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.51
accuracy: 80.66%


# MLPClassifier (for comparison purposes)

In [114]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

mlp1 = MLPClassifier(hidden_layer_sizes=(50,50,50,50), 
                     max_iter=1000,   
                     solver='adam',
                     alpha=.1
                     )

mlp1.fit(train_x, train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


MLPClassifier(alpha=0.1, hidden_layer_sizes=(50, 50, 50, 50), max_iter=1000)

In [115]:
#Predict the train values
train_y_pred = mlp1.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.9275151515151515

In [116]:
#Predict the test values
test_y_pred = mlp1.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8410633484162896